In [1]:
pip install cryptography

Note: you may need to restart the kernel to use updated packages.


In [2]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

def generate_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
    )
    public_key = private_key.public_key()

    # Save the private key
    with open("private_key.pem", "wb") as private_key_file:
        private_key_file.write(
            private_key.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.TraditionalOpenSSL,
                encryption_algorithm=serialization.NoEncryption()
            )
        )

    # Save the public key
    with open("public_key.pem", "wb") as public_key_file:
        public_key_file.write(
            public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            )
        )

generate_keys()

In [3]:
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

def encrypt_file(file_path, public_key_path):
    with open(file_path, "rb") as file:
        file_data = file.read()

    with open(public_key_path, "rb") as key_file:
        public_key = serialization.load_pem_public_key(key_file.read())

    encrypted_data = public_key.encrypt(
        file_data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    with open(f"{file_path}.enc", "wb") as encrypted_file:
        encrypted_file.write(encrypted_data)

encrypt_file("file.txt", "public_key.pem")

In [4]:
import os

def encrypt_file(file_path, public_key_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    if not os.path.exists(public_key_path):
        raise FileNotFoundError(f"The public key file {public_key_path} does not exist.")

    with open(file_path, "rb") as file:
        file_data = file.read()

    with open(public_key_path, "rb") as key_file:
        public_key = key_file.read()

    # Add encryption logic here (e.g., using cryptography library)

    encrypted_data = b"Encrypted content"  # Placeholder
    with open(f"{file_path}.enc", "wb") as encrypted_file:
        encrypted_file.write(encrypted_data)

In [5]:
def decrypt_file(encrypted_file_path, private_key_path):
    with open(encrypted_file_path, "rb") as encrypted_file:
        encrypted_data = encrypted_file.read()

    with open(private_key_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(key_file.read(), password=None)

    decrypted_data = private_key.decrypt(
        encrypted_data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    with open(f"{encrypted_file_path}.dec", "wb") as decrypted_file:
        decrypted_file.write(decrypted_data)

decrypt_file("file.txt.enc", "private_key.pem")